<center>
    <h1> 
        Treinamento das Redes Neurais para Classificação dos Histogramas
    </h1>
</center>
    <div style="text-align: right;"><h3>
        Carlos Eduardo Cassimiro da Silva
    </h3></div>
<br>
<p>Após desenvolver e executar o pipeline do pré-processamento dos dados, podemos iniciar o treinamento e validação preliminar dos modelos de classificação. </p>
<p>Existem diferentes classificações que podemos tentar realizar dependendo da abstração escolhida para separar os documentos:  <br>
1. Separar os documentos entre os abertos e os "normais" (frente e verso), e depois classificá-los em suas respectivas categorias; <br>
1.1. Categorizar os documentos no seu respectivo tipo (CNH, RG, CPF); <br>
1.2. Categorizar os documentos no seu respectivo tipo e também segmentantá-los (CNH Frente, CNH Verso, RG Frente e etc); <br>
2. Categorizar os documentos no seu respectivo tipo (CNH, RG, CPF), sem distinguir entre abertos e "normais".
</p> 
<p>Além disso, podemos utilizar como entrada dos modelos diferentes atributos: <br>
1. A junção dos histogramas dos canais de cores do padrão RGB; <br>
2. A junção dos histogramas dos canais de cores do padrão RGB + os 100 valores do histograma da matriz(hue) do padrão HSV; <br>
3. Os 100 valores do histograma da matriz(hue) do padrão HSV.
</p>
<p>Nesse notebook iremos treinar e verificar a efetividade no desenvolvimento dos modelos para os diferentes arranjos entre os inputs e os targets, e eleger os modelos mais promissores para a validação final. A avaliação dos testes estará compilada nas conclusões</p>

<h4>Desafio #1: Classificação de documentos (RG, CNH e CPF) </h4>

 - Contextualização: Inúmeras áreas de diferentes organizações (usualmente como parte de um processo de um backoffice) recepcionam documentos dos seus clientes para formação de kits de documentação. Tais kits são, por exemplo, compartilhados com outros stakeholders das empresas. Conforme pode-se pressupor, um desafio nesse cenário refere-se ao fato de que o cliente pode enviar documentos, porém sem necessariamente indicar a qual tipo se o documento se refere (RG, CNH ou CPF, por exemplo). Dessa forma, ao invés de demandar um trabalho manual para essa leitura e classificação dos documentos, podemos construir um modelo de aprendizado que tenha capacidade de ler um conjunto de documentos (em .jpg, por exemplo) e, subsequentemente, realizar a classificação em três tipos distintos: RG, CNH e CPF.
 - Dataset: Para esse desafio utilizaremos um dataset público de RG, CNH e CPF (incluindo as imagens). Este repositório apresenta o conjunto de dados denominado Brazilian Identity Document Dataset (BID Dataset), o primeiro conjunto de dados público de documentos de identificação brasileiros. <br>
Link do dataset: https://github.com/ricardobnjunior/Brazilian-Identity-Document-Dataset

<h4>Roteiro</h4>
<b>Módulos utilizados</b> <br>
<b>1. Teste 1: Sem as amostras grandes e com 6 labels</b><br>
1.1 Modelo 1 (1 camada oculta densa)<br>
1.2 Modelo 2 (2 camadas ocultas densas)<br>
<b>2. Teste 2: Sem as amostras grandes e com 3 labels</b><br>
2.1 Modelo 1 (1 camada oculta densa)<br>
2.2 Modelo 2 (2 camadas ocultas densas)<br>
2.3 Modelo 3 (2 camadas ocultas: 1 densa, 1 BatchNormalization) <br>
<b>3. Teste 3: apenas os 100 valores do HSV, 3 classes e sem as amostras grandes</b><br>
3.1 Modelo 3 (2 camadas ocultas: 1 densa, 1 BatchNormalization)<br>
3.2 Modelo 3* (2 camadas ocultas: 1 densa, 1 BatchNormalization)<br>
<b>4. Teste 4 com os documentos grandes</b><br>
4.1 Modelo 3 (3 camadas ocultas: 1 densa, 1 BatchNormalization, 1 densa)<br>
4.2 Modelo 2 (2 camadas ocultas densas)<br>
<b>5. Teste 5: 100 valores do hsv, com amostras grandes e 3 classes</b><br>
5.1 Modelo 3 (2 camadas ocultas: 1 densa, 1 BatchNormalization)<br>
5.2 Modelo 3* (2 camadas ocultas: 1 densa, 1 BatchNormalization)<br>
<b>6. Teste 6 apenas com o RGB</b><br>
6.1 Modelo 3 (2 camadas ocultas: 1 densa, 1 BatchNormalization)<br>
6.2 Modelo 4 (3 camadas ocultas: 1 densa, 1 BatchNormalization, 1 densa) <br>
<b>7. Conclusão</b>

In [1]:
import keras.backend as K
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout
import numpy as np
import json
from sklearn.model_selection import train_test_split
from copy import deepcopy

In [4]:
# Abrindo o conjunto de dados (transformados)
with open('imagens_originais/desafio1_data.json') as file: 
    data = json.load(file)

# 1. Teste 1: Sem as amostras grandes e com 6 labels

In [ ]:
# Copiando o conjunto de teinamento e os targets
X_op2 = deepcopy(data['op2'])
y_7 = deepcopy(data['y7'])

In [6]:
len(y_7)

28800

In [7]:
# Retirando os documentos abertos (grandes)
y_7_mod = deepcopy(y_7[3600:])
X_op2_mod = deepcopy(X_op2[3600:])

In [8]:
len(y_7_mod)

25200

In [9]:
# Terminando de remover os documentos abertos
del y_7_mod[14400:18000]
del X_op2_mod[14400:18000]

In [10]:
# Verificando se ainda há documentos abertos (grandes)
for i in range(len(y_7_mod)) :
    if y_7_mod[i] == 'GRANDE':
        print(i)

In [11]:
len(y_7_mod)

21600

In [12]:
# Tranformando em numpy.arrays
y_7_mod = np.array(y_7_mod)
X_op2_mod = np.array(X_op2_mod)

In [13]:
# Verificando as labels dos targets
np.unique(y_7_mod)

array(['CNH_Frente', 'CNH_Verso', 'CPF_Frente', 'CPF_Verso', 'RG_Frente',
       'RG_Verso'], dtype='<U10')

In [14]:
# Transformando as labels dos targets em números
labels = {}
for i,j in zip(np.unique(y_7_mod), range(len(np.unique(y_7_mod)))):
    labels[i] = j

In [15]:
labels

{'CNH_Frente': 0,
 'CNH_Verso': 1,
 'CPF_Frente': 2,
 'CPF_Verso': 3,
 'RG_Frente': 4,
 'RG_Verso': 5}

In [16]:
X_op2_mod[1].shape

(850,)

In [17]:
# Copiando os targets em formato de número para outra lista
y_7_label = deepcopy(y_7_mod)
for i in range(len(y_7_mod)):
    y_7_label[i] = labels[y_7_mod[i]]

In [18]:
# Codificando os targets para o padrão que o keras aceita sem precisar mudar parâmetros
y_7_label = to_categorical(y_7_label,len(np.unique(y_7_label)))

In [19]:
# Verificando a codificação
y_7_label

array([[1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.]], dtype=float32)

In [20]:
# Dividindo o conjunto de dados em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X_op2_mod, y_7_label, test_size=0.3)

In [21]:
X_op2_mod[0].shape

(850,)

## 1.1 Modelo 1 (1 camada oculta densa)

In [22]:
# Criando uma rede neural
model = Sequential() # Definindo o tipo da rede neural
model.add(Dense(50, activation='relu', input_shape=(X_op2_mod[0].shape))) # Definindo a primeira cada oculta
model.add(Dropout(.25)) # Definindo um dropout para retirar os atributos menos relevantes 
model.add(Dense(6, activation='softmax')) # Definindo a saída
model.compile(optimizer="rmsprop", loss="categorical_crossentropy",metrics=['categorical_accuracy']) 
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                42550     
                                                                 
 dropout (Dropout)           (None, 50)                0         
                                                                 
 dense_1 (Dense)             (None, 6)                 306       
                                                                 
Total params: 42,856
Trainable params: 42,856
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Executando o treinamento e teste
model.fit(X_train, y_train, validation_data=[X_test, y_test], epochs=20, batch_size=32, verbose=1)

Epoch 1/20
473/473 [==============================] - 2s 2ms/step - loss: 0.8853 - categorical_accuracy: 0.6911 - val_loss: 0.6267 - val_categorical_accuracy: 0.7894
Epoch 2/20
473/473 [==============================] - 1s 1ms/step - loss: 0.5825 - categorical_accuracy: 0.7974 - val_loss: 0.5149 - val_categorical_accuracy: 0.8184
Epoch 3/20
473/473 [==============================] - 1s 1ms/step - loss: 0.5018 - categorical_accuracy: 0.8156 - val_loss: 0.4527 - val_categorical_accuracy: 0.8307
Epoch 4/20
473/473 [==============================] - 1s 1ms/step - loss: 0.4612 - categorical_accuracy: 0.8276 - val_loss: 0.4350 - val_categorical_accuracy: 0.8279
Epoch 5/20
473/473 [==============================] - 1s 1ms/step - loss: 0.4359 - categorical_accuracy: 0.8336 - val_loss: 0.4116 - val_categorical_accuracy: 0.8455
Epoch 6/20
473/473 [==============================] - 1s 1ms/step - loss: 0.4181 - categorical_accuracy: 0.8401 - val_loss: 0.3943 - val_categorical_accuracy: 0.8503
Epoc

## 1.2 Modelo 2 (2 camadas ocultas densas)

Os procedimentos sequintes serão análogos aos anteiores.

In [24]:
model = Sequential()
model.add(Dense(425, activation='relu', input_shape=(X_op2_mod[0].shape)))
model.add(Dropout(.25))
model.add(Dense(25, activation='relu'))
model.add(Dropout(.25))
model.add(Dense(6, activation='softmax'))
model.compile(optimizer="rmsprop", loss="categorical_crossentropy",metrics=['categorical_accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 425)               361675    
                                                                 
 dropout_1 (Dropout)         (None, 425)               0         
                                                                 
 dense_3 (Dense)             (None, 25)                10650     
                                                                 
 dropout_2 (Dropout)         (None, 25)                0         
                                                                 
 dense_4 (Dense)             (None, 6)                 156       
                                                                 
Total params: 372,481
Trainable params: 372,481
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.fit(X_train, y_train, validation_data=[X_test, y_test], epochs=20, batch_size=32, verbose=1)

Epoch 1/20
473/473 [==============================] - 3s 6ms/step - loss: 0.8022 - categorical_accuracy: 0.7049 - val_loss: 0.4792 - val_categorical_accuracy: 0.8194
Epoch 2/20
473/473 [==============================] - 3s 6ms/step - loss: 0.5065 - categorical_accuracy: 0.8095 - val_loss: 0.4263 - val_categorical_accuracy: 0.8427
Epoch 3/20
473/473 [==============================] - 3s 5ms/step - loss: 0.4372 - categorical_accuracy: 0.8317 - val_loss: 0.3813 - val_categorical_accuracy: 0.8540
Epoch 4/20
473/473 [==============================] - 3s 6ms/step - loss: 0.4032 - categorical_accuracy: 0.8475 - val_loss: 0.3656 - val_categorical_accuracy: 0.8594
Epoch 5/20
473/473 [==============================] - 3s 6ms/step - loss: 0.3823 - categorical_accuracy: 0.8555 - val_loss: 0.3644 - val_categorical_accuracy: 0.8670
Epoch 6/20
473/473 [==============================] - 3s 6ms/step - loss: 0.3593 - categorical_accuracy: 0.8633 - val_loss: 0.3346 - val_categorical_accuracy: 0.8809
Epoc

# 2. Teste 2: Sem as amostras grandes e com 3 labels

In [26]:
labels_3 = {'CNH_Frente': 0,
            'CNH_Verso': 0,
            'CPF_Frente': 1,
            'CPF_Verso': 1,
            'RG_Frente': 2,
            'RG_Verso': 2}

In [27]:
y_7_label3 = deepcopy(y_7_mod)
for i in range(len(y_7_mod)):
    y_7_label3[i] = labels_3[y_7_mod[i]]

In [28]:
y_7_label3 = to_categorical(y_7_label3,len(np.unique(y_7_label3)))

In [29]:
y_7_label

array([[1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.]], dtype=float32)

In [30]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_op2_mod, y_7_label3, test_size=0.3)

## 2.1 Modelo 1 (1 camada oculta densa)

In [31]:
model = Sequential()
model.add(Dense(50, activation='relu', input_shape=(X_op2_mod[0].shape)))
model.add(Dropout(.25))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer="rmsprop", loss="categorical_crossentropy",metrics=['categorical_accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 50)                42550     
                                                                 
 dropout_3 (Dropout)         (None, 50)                0         
                                                                 
 dense_6 (Dense)             (None, 3)                 153       
                                                                 
Total params: 42,703
Trainable params: 42,703
Non-trainable params: 0
_________________________________________________________________


In [32]:
model.fit(X_train2, y_train2, validation_data=[X_test2, y_test2], epochs=20, batch_size=32, verbose=1)

Epoch 1/20
473/473 [==============================] - 1s 1ms/step - loss: 0.3268 - categorical_accuracy: 0.9014 - val_loss: 0.1908 - val_categorical_accuracy: 0.9403
Epoch 2/20
473/473 [==============================] - 1s 1ms/step - loss: 0.1770 - categorical_accuracy: 0.9478 - val_loss: 0.1503 - val_categorical_accuracy: 0.9560
Epoch 3/20
473/473 [==============================] - 1s 1ms/step - loss: 0.1507 - categorical_accuracy: 0.9534 - val_loss: 0.1209 - val_categorical_accuracy: 0.9636
Epoch 4/20
473/473 [==============================] - 1s 1ms/step - loss: 0.1326 - categorical_accuracy: 0.9606 - val_loss: 0.1200 - val_categorical_accuracy: 0.9619
Epoch 5/20
473/473 [==============================] - 1s 1ms/step - loss: 0.1217 - categorical_accuracy: 0.9634 - val_loss: 0.1042 - val_categorical_accuracy: 0.9687
Epoch 6/20
473/473 [==============================] - 1s 1ms/step - loss: 0.1166 - categorical_accuracy: 0.9660 - val_loss: 0.1034 - val_categorical_accuracy: 0.9659
Epoc

## 2.2 Modelo 2 (2 camadas ocultas densas)

In [33]:
model = Sequential()
model.add(Dense(425, activation='relu', input_shape=(X_op2_mod[0].shape)))
model.add(Dropout(.25))
model.add(Dense(25, activation='relu'))
model.add(Dropout(.25))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer="rmsprop", loss="categorical_crossentropy",metrics=['categorical_accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 425)               361675    
                                                                 
 dropout_4 (Dropout)         (None, 425)               0         
                                                                 
 dense_8 (Dense)             (None, 25)                10650     
                                                                 
 dropout_5 (Dropout)         (None, 25)                0         
                                                                 
 dense_9 (Dense)             (None, 3)                 78        
                                                                 
Total params: 372,403
Trainable params: 372,403
Non-trainable params: 0
_________________________________________________________________


In [34]:
model.fit(X_train2, y_train2, validation_data=[X_test2, y_test2], epochs=20, batch_size=32, verbose=1)

Epoch 1/20
473/473 [==============================] - 3s 6ms/step - loss: 0.2699 - categorical_accuracy: 0.9118 - val_loss: 0.1371 - val_categorical_accuracy: 0.9566
Epoch 2/20
473/473 [==============================] - 3s 6ms/step - loss: 0.1533 - categorical_accuracy: 0.9510 - val_loss: 0.1145 - val_categorical_accuracy: 0.9647
Epoch 3/20
473/473 [==============================] - 3s 6ms/step - loss: 0.1228 - categorical_accuracy: 0.9617 - val_loss: 0.0988 - val_categorical_accuracy: 0.9673
Epoch 4/20
473/473 [==============================] - 3s 6ms/step - loss: 0.1147 - categorical_accuracy: 0.9659 - val_loss: 0.1093 - val_categorical_accuracy: 0.9681
Epoch 5/20
473/473 [==============================] - 3s 6ms/step - loss: 0.1075 - categorical_accuracy: 0.9697 - val_loss: 0.0899 - val_categorical_accuracy: 0.9735
Epoch 6/20
473/473 [==============================] - 3s 6ms/step - loss: 0.1034 - categorical_accuracy: 0.9714 - val_loss: 0.0969 - val_categorical_accuracy: 0.9687
Epoc

## 2.3 Modelo 3 (2 camadas ocultas: 1 densa, 1 BatchNormalization) 

In [35]:
from keras.layers import LSTM

In [36]:
from keras.layers import GRU

In [37]:
from keras.layers import BatchNormalization

In [38]:
model = Sequential()
model.add(Dense(50, activation='relu', input_shape=(X_op2_mod[0].shape)))
model.add(BatchNormalization())
model.add(Dropout(.25))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer="rmsprop", loss="categorical_crossentropy",metrics=['categorical_accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 50)                42550     
                                                                 
 batch_normalization (BatchN  (None, 50)               200       
 ormalization)                                                   
                                                                 
 dropout_6 (Dropout)         (None, 50)                0         
                                                                 
 dense_11 (Dense)            (None, 3)                 153       
                                                                 
Total params: 42,903
Trainable params: 42,803
Non-trainable params: 100
_________________________________________________________________


In [39]:
model.fit(X_train2, y_train2, validation_data=[X_test2, y_test2], epochs=20, batch_size=32, verbose=1)

Epoch 1/20
473/473 [==============================] - 2s 1ms/step - loss: 0.2691 - categorical_accuracy: 0.9111 - val_loss: 0.1594 - val_categorical_accuracy: 0.9520
Epoch 2/20
473/473 [==============================] - 1s 1ms/step - loss: 0.1539 - categorical_accuracy: 0.9492 - val_loss: 0.1148 - val_categorical_accuracy: 0.9620
Epoch 3/20
473/473 [==============================] - 1s 1ms/step - loss: 0.1372 - categorical_accuracy: 0.9547 - val_loss: 0.0985 - val_categorical_accuracy: 0.9693
Epoch 4/20
473/473 [==============================] - 1s 1ms/step - loss: 0.1229 - categorical_accuracy: 0.9613 - val_loss: 0.0898 - val_categorical_accuracy: 0.9708
Epoch 5/20
473/473 [==============================] - 1s 1ms/step - loss: 0.1218 - categorical_accuracy: 0.9618 - val_loss: 0.0936 - val_categorical_accuracy: 0.9708
Epoch 6/20
473/473 [==============================] - 1s 1ms/step - loss: 0.1134 - categorical_accuracy: 0.9629 - val_loss: 0.0927 - val_categorical_accuracy: 0.9718
Epoc

# 3. Teste 3 : apenas os 100 valores do HSV, 3 classes e sem as amostras grandes

In [41]:
X_op2_mod2 = deepcopy(X_op2_mod)

In [49]:
X_op2_mod2 = []
for i in range(len(X_op2_mod)):
    X_op2_mod2.append(X_op2_mod[i][-100:])

In [50]:
X_train3, X_test3, y_train3, y_test3 = train_test_split(X_op2_mod2, y_7_label3, test_size=0.3)

## 3.1 Modelo 3 (2 camadas ocultas: 1 densa, 1 BatchNormalization) 

In [54]:
model = Sequential()
model.add(Dense(10, activation='relu', input_shape=(X_op2_mod2[0].shape)))
model.add(BatchNormalization())
model.add(Dropout(.25))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer="rmsprop", loss="categorical_crossentropy",metrics=['categorical_accuracy'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 10)                1010      
                                                                 
 batch_normalization_2 (Batc  (None, 10)               40        
 hNormalization)                                                 
                                                                 
 dropout_8 (Dropout)         (None, 10)                0         
                                                                 
 dense_15 (Dense)            (None, 3)                 33        
                                                                 
Total params: 1,083
Trainable params: 1,063
Non-trainable params: 20
_________________________________________________________________


In [59]:
model.fit(np.array(X_train3), np.array(y_train3), validation_data=[np.array(X_test3), np.array(y_test3)], epochs=20, batch_size=32, verbose=1)

Epoch 1/20
473/473 [==============================] - 2s 2ms/step - loss: 0.4049 - categorical_accuracy: 0.8665 - val_loss: 0.2514 - val_categorical_accuracy: 0.9307
Epoch 2/20
473/473 [==============================] - 1s 2ms/step - loss: 0.2655 - categorical_accuracy: 0.9146 - val_loss: 0.1858 - val_categorical_accuracy: 0.9398
Epoch 3/20
473/473 [==============================] - 1s 2ms/step - loss: 0.2218 - categorical_accuracy: 0.9296 - val_loss: 0.1637 - val_categorical_accuracy: 0.9454
Epoch 4/20
473/473 [==============================] - 1s 2ms/step - loss: 0.2012 - categorical_accuracy: 0.9356 - val_loss: 0.1570 - val_categorical_accuracy: 0.9497
Epoch 5/20
473/473 [==============================] - 1s 2ms/step - loss: 0.2030 - categorical_accuracy: 0.9368 - val_loss: 0.1504 - val_categorical_accuracy: 0.9517
Epoch 6/20
473/473 [==============================] - 1s 2ms/step - loss: 0.1967 - categorical_accuracy: 0.9380 - val_loss: 0.1547 - val_categorical_accuracy: 0.9497
Epoc

## 3.2 Modelo 3* (2 camadas ocultas: 1 densa, 1 BatchNormalization)

In [60]:
model = Sequential()
model.add(Dense(20, activation='relu', input_shape=(X_op2_mod2[0].shape)))
model.add(BatchNormalization())
model.add(Dropout(.25))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer="rmsprop", loss="categorical_crossentropy",metrics=['categorical_accuracy'])
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 20)                2020      
                                                                 
 batch_normalization_3 (Batc  (None, 20)               80        
 hNormalization)                                                 
                                                                 
 dropout_9 (Dropout)         (None, 20)                0         
                                                                 
 dense_17 (Dense)            (None, 3)                 63        
                                                                 
Total params: 2,163
Trainable params: 2,123
Non-trainable params: 40
_________________________________________________________________


In [61]:
model.fit(np.array(X_train3), np.array(y_train3), validation_data=[np.array(X_test3), np.array(y_test3)], epochs=20, batch_size=32, verbose=1)

Epoch 1/20
473/473 [==============================] - 2s 2ms/step - loss: 0.3094 - categorical_accuracy: 0.8988 - val_loss: 0.1967 - val_categorical_accuracy: 0.9452
Epoch 2/20
473/473 [==============================] - 1s 2ms/step - loss: 0.1878 - categorical_accuracy: 0.9419 - val_loss: 0.1410 - val_categorical_accuracy: 0.9556
Epoch 3/20
473/473 [==============================] - 1s 2ms/step - loss: 0.1695 - categorical_accuracy: 0.9491 - val_loss: 0.1343 - val_categorical_accuracy: 0.9585
Epoch 4/20
473/473 [==============================] - 1s 2ms/step - loss: 0.1523 - categorical_accuracy: 0.9530 - val_loss: 0.1263 - val_categorical_accuracy: 0.9600
Epoch 5/20
473/473 [==============================] - 1s 2ms/step - loss: 0.1456 - categorical_accuracy: 0.9563 - val_loss: 0.1133 - val_categorical_accuracy: 0.9651
Epoch 6/20
473/473 [==============================] - 1s 2ms/step - loss: 0.1492 - categorical_accuracy: 0.9532 - val_loss: 0.1146 - val_categorical_accuracy: 0.9633
Epoc

# 4. Teste 4 com os documentos grandes

In [68]:
y_3 = deepcopy(data['y3'])

In [69]:
labels_3 = {}
for i,j in zip(np.unique(y_3), range(len(np.unique(y_3)))):
    labels_3[i] = j

In [70]:
labels_3

{'CNH': 0, 'CPF': 1, 'RG': 2}

In [71]:
for i in range(len(y_3)):
    y_3[i] = labels_3[y_3[i]]

In [72]:
y_3 = to_categorical(y_3,len(np.unique(y_3)))

## 4.1 Modelo 3 (3 camadas ocultas: 1 densa, 1 BatchNormalization, 1 densa)

In [78]:
X_train4, X_test4, y_train4, y_test4 = train_test_split(X_op2, y_3, test_size=0.3)

In [85]:
model = Sequential()
model.add(Dense(50, activation='relu', input_shape=(np.array(X_op2)[0].shape)))
model.add(BatchNormalization())
model.add(Dropout(.25))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer="rmsprop", loss="categorical_crossentropy",metrics=['categorical_accuracy'])
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 50)                42550     
                                                                 
 batch_normalization_4 (Batc  (None, 50)               200       
 hNormalization)                                                 
                                                                 
 dropout_14 (Dropout)        (None, 50)                0         
                                                                 
 dense_25 (Dense)            (None, 3)                 153       
                                                                 
Total params: 42,903
Trainable params: 42,803
Non-trainable params: 100
_________________________________________________________________


In [86]:
model.fit(np.array(X_train4), np.array(y_train4), validation_data=[np.array(X_test4), np.array(y_test4)], epochs=20, batch_size=32, verbose=1)

Epoch 1/20
630/630 [==============================] - 2s 2ms/step - loss: 0.2494 - categorical_accuracy: 0.9216 - val_loss: 0.1255 - val_categorical_accuracy: 0.9637
Epoch 2/20
630/630 [==============================] - 1s 2ms/step - loss: 0.1346 - categorical_accuracy: 0.9587 - val_loss: 0.1073 - val_categorical_accuracy: 0.9663
Epoch 3/20
630/630 [==============================] - 1s 2ms/step - loss: 0.1218 - categorical_accuracy: 0.9623 - val_loss: 0.1024 - val_categorical_accuracy: 0.9693
Epoch 4/20
630/630 [==============================] - 1s 2ms/step - loss: 0.1173 - categorical_accuracy: 0.9628 - val_loss: 0.0857 - val_categorical_accuracy: 0.9742
Epoch 5/20
630/630 [==============================] - 1s 2ms/step - loss: 0.1117 - categorical_accuracy: 0.9664 - val_loss: 0.0917 - val_categorical_accuracy: 0.9719
Epoch 6/20
630/630 [==============================] - 1s 1ms/step - loss: 0.1024 - categorical_accuracy: 0.9676 - val_loss: 0.0865 - val_categorical_accuracy: 0.9721
Epoc

## 4.2 Modelo 2 (2 camadas ocultas densas)

In [83]:
model = Sequential()
model.add(Dense(425, activation='relu', input_shape=(np.array(X_op2)[0].shape)))
model.add(Dropout(.25))
model.add(Dense(25, activation='relu'))
model.add(Dropout(.25))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer="rmsprop", loss="categorical_crossentropy",metrics=['categorical_accuracy'])
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 425)               361675    
                                                                 
 dropout_12 (Dropout)        (None, 425)               0         
                                                                 
 dense_22 (Dense)            (None, 25)                10650     
                                                                 
 dropout_13 (Dropout)        (None, 25)                0         
                                                                 
 dense_23 (Dense)            (None, 3)                 78        
                                                                 
Total params: 372,403
Trainable params: 372,403
Non-trainable params: 0
_________________________________________________________________


In [84]:
model.fit(np.array(X_train4), np.array(y_train4), validation_data=[np.array(X_test4), np.array(y_test4)], epochs=20, batch_size=32, verbose=1)

Epoch 1/20
630/630 [==============================] - 5s 7ms/step - loss: 0.2405 - categorical_accuracy: 0.9203 - val_loss: 0.1286 - val_categorical_accuracy: 0.9578
Epoch 2/20
630/630 [==============================] - 3s 5ms/step - loss: 0.1321 - categorical_accuracy: 0.9591 - val_loss: 0.1029 - val_categorical_accuracy: 0.9706
Epoch 3/20
630/630 [==============================] - 3s 5ms/step - loss: 0.1184 - categorical_accuracy: 0.9659 - val_loss: 0.0939 - val_categorical_accuracy: 0.9740
Epoch 4/20
630/630 [==============================] - 3s 5ms/step - loss: 0.1088 - categorical_accuracy: 0.9700 - val_loss: 0.0993 - val_categorical_accuracy: 0.9719
Epoch 5/20
630/630 [==============================] - 3s 5ms/step - loss: 0.0996 - categorical_accuracy: 0.9731 - val_loss: 0.1071 - val_categorical_accuracy: 0.9745
Epoch 6/20
630/630 [==============================] - 3s 5ms/step - loss: 0.0961 - categorical_accuracy: 0.9731 - val_loss: 0.0918 - val_categorical_accuracy: 0.9759
Epoc

# 5. Teste 5: 100 valores do hsv, com amostras grandes e 3 classes

In [92]:
X_op2_mod3 = []
for i in range(len(X_op2)):
    X_op2_mod3.append(X_op2[i][-100:])

In [96]:
X_train5, X_test5, y_train5, y_test5 = train_test_split(X_op2_mod3, y_3, test_size=0.3)

## 5.1 Modelo 3 (2 camadas ocultas: 1 densa, 1 BatchNormalization)

In [97]:
X_op2_mod3 = np.array(X_op2_mod3)

In [98]:
model = Sequential()
model.add(Dense(10, activation='relu', input_shape=(X_op2_mod3[0].shape)))
model.add(BatchNormalization())
model.add(Dropout(.25))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer="rmsprop", loss="categorical_crossentropy",metrics=['categorical_accuracy'])
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_26 (Dense)            (None, 10)                1010      
                                                                 
 batch_normalization_5 (Batc  (None, 10)               40        
 hNormalization)                                                 
                                                                 
 dropout_15 (Dropout)        (None, 10)                0         
                                                                 
 dense_27 (Dense)            (None, 3)                 33        
                                                                 
Total params: 1,083
Trainable params: 1,063
Non-trainable params: 20
_________________________________________________________________


In [99]:
model.fit(np.array(X_train5), np.array(y_train5), validation_data=[np.array(X_test5), np.array(y_test5)], epochs=20, batch_size=32, verbose=1)

Epoch 1/20
630/630 [==============================] - 2s 2ms/step - loss: 0.3534 - categorical_accuracy: 0.8920 - val_loss: 0.1712 - val_categorical_accuracy: 0.9485
Epoch 2/20
630/630 [==============================] - 1s 2ms/step - loss: 0.2098 - categorical_accuracy: 0.9362 - val_loss: 0.1482 - val_categorical_accuracy: 0.9532
Epoch 3/20
630/630 [==============================] - 1s 2ms/step - loss: 0.1969 - categorical_accuracy: 0.9415 - val_loss: 0.1328 - val_categorical_accuracy: 0.9560
Epoch 4/20
630/630 [==============================] - 1s 2ms/step - loss: 0.1887 - categorical_accuracy: 0.9415 - val_loss: 0.1299 - val_categorical_accuracy: 0.9566
Epoch 5/20
630/630 [==============================] - 1s 2ms/step - loss: 0.1816 - categorical_accuracy: 0.9456 - val_loss: 0.1279 - val_categorical_accuracy: 0.9550
Epoch 6/20
630/630 [==============================] - 1s 2ms/step - loss: 0.1769 - categorical_accuracy: 0.9457 - val_loss: 0.1189 - val_categorical_accuracy: 0.9590
Epoc

## 5.2 Modelo 3* (2 camadas ocultas: 1 densa, 1 BatchNormalization)

In [100]:
model = Sequential()
model.add(Dense(20, activation='relu', input_shape=(X_op2_mod3[0].shape)))
model.add(BatchNormalization())
model.add(Dropout(.25))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer="rmsprop", loss="categorical_crossentropy",metrics=['categorical_accuracy'])
model.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_28 (Dense)            (None, 20)                2020      
                                                                 
 batch_normalization_6 (Batc  (None, 20)               80        
 hNormalization)                                                 
                                                                 
 dropout_16 (Dropout)        (None, 20)                0         
                                                                 
 dense_29 (Dense)            (None, 3)                 63        
                                                                 
Total params: 2,163
Trainable params: 2,123
Non-trainable params: 40
_________________________________________________________________


In [101]:
model.fit(np.array(X_train5), np.array(y_train5), validation_data=[np.array(X_test5), np.array(y_test5)], epochs=20, batch_size=32, verbose=1)

Epoch 1/20
630/630 [==============================] - 1s 1ms/step - loss: 0.2935 - categorical_accuracy: 0.9043 - val_loss: 0.1431 - val_categorical_accuracy: 0.9572
Epoch 2/20
630/630 [==============================] - 1s 894us/step - loss: 0.1719 - categorical_accuracy: 0.9464 - val_loss: 0.1164 - val_categorical_accuracy: 0.9628
Epoch 3/20
630/630 [==============================] - 1s 909us/step - loss: 0.1597 - categorical_accuracy: 0.9538 - val_loss: 0.1046 - val_categorical_accuracy: 0.9675
Epoch 4/20
630/630 [==============================] - 1s 919us/step - loss: 0.1497 - categorical_accuracy: 0.9546 - val_loss: 0.0980 - val_categorical_accuracy: 0.9691
Epoch 5/20
630/630 [==============================] - 1s 901us/step - loss: 0.1415 - categorical_accuracy: 0.9567 - val_loss: 0.1051 - val_categorical_accuracy: 0.9664
Epoch 6/20
630/630 [==============================] - 1s 930us/step - loss: 0.1376 - categorical_accuracy: 0.9583 - val_loss: 0.0913 - val_categorical_accuracy: 0

# 6. Teste 6 apenas com o RGB

In [103]:
X_opt1 = deepcopy(data['opt1'])

In [104]:
X_train6, X_test6, y_train6, y_test6 = train_test_split(X_opt1, y_3, test_size=0.3)

## 6.1 Modelo 3 (2 camadas ocultas: 1 densa, 1 BatchNormalization) 

In [107]:
X_opt1 = np.array(X_opt1)

In [108]:
model = Sequential()
model.add(Dense(50, activation='relu', input_shape=(X_opt1[0].shape)))
model.add(BatchNormalization())
model.add(Dropout(.25))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer="rmsprop", loss="categorical_crossentropy",metrics=['categorical_accuracy'])
model.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 50)                37550     
                                                                 
 batch_normalization_7 (Batc  (None, 50)               200       
 hNormalization)                                                 
                                                                 
 dropout_17 (Dropout)        (None, 50)                0         
                                                                 
 dense_31 (Dense)            (None, 3)                 153       
                                                                 
Total params: 37,903
Trainable params: 37,803
Non-trainable params: 100
_________________________________________________________________


In [109]:
model.fit(np.array(X_train6), np.array(y_train6), validation_data=[np.array(X_test6), np.array(y_test6)], epochs=20, batch_size=32, verbose=1)

Epoch 1/20
630/630 [==============================] - 2s 2ms/step - loss: 0.7938 - categorical_accuracy: 0.6292 - val_loss: 0.6604 - val_categorical_accuracy: 0.6955
Epoch 2/20
630/630 [==============================] - 1s 2ms/step - loss: 0.6798 - categorical_accuracy: 0.6765 - val_loss: 0.6066 - val_categorical_accuracy: 0.7135
Epoch 3/20
630/630 [==============================] - 1s 2ms/step - loss: 0.6583 - categorical_accuracy: 0.6839 - val_loss: 0.6059 - val_categorical_accuracy: 0.7104
Epoch 4/20
630/630 [==============================] - 1s 2ms/step - loss: 0.6460 - categorical_accuracy: 0.6876 - val_loss: 0.5942 - val_categorical_accuracy: 0.7137
Epoch 5/20
630/630 [==============================] - 1s 2ms/step - loss: 0.6409 - categorical_accuracy: 0.6918 - val_loss: 0.5947 - val_categorical_accuracy: 0.7211
Epoch 6/20
630/630 [==============================] - 1s 1ms/step - loss: 0.6382 - categorical_accuracy: 0.6950 - val_loss: 0.5893 - val_categorical_accuracy: 0.7098
Epoc

## 6.2 Modelo 4 (3 camadas ocultas: 1 densa, 1 BatchNormalization, 1 densa) 

In [110]:
model = Sequential()
model.add(Dense(375, activation='relu', input_shape=(X_opt1[0].shape)))
model.add(BatchNormalization())
model.add(Dropout(.25))
model.add(Dense(25, activation='relu'))
model.add(Dropout(.25))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer="rmsprop", loss="categorical_crossentropy",metrics=['categorical_accuracy'])
model.summary()

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_32 (Dense)            (None, 375)               281625    
                                                                 
 batch_normalization_8 (Batc  (None, 375)              1500      
 hNormalization)                                                 
                                                                 
 dropout_18 (Dropout)        (None, 375)               0         
                                                                 
 dense_33 (Dense)            (None, 25)                9400      
                                                                 
 dropout_19 (Dropout)        (None, 25)                0         
                                                                 
 dense_34 (Dense)            (None, 3)                 78        
                                                     

In [111]:
model.fit(np.array(X_train6), np.array(y_train6), validation_data=[np.array(X_test6), np.array(y_test6)], epochs=20, batch_size=32, verbose=1)

Epoch 1/20
630/630 [==============================] - 4s 6ms/step - loss: 0.7672 - categorical_accuracy: 0.6434 - val_loss: 0.6131 - val_categorical_accuracy: 0.7001
Epoch 2/20
630/630 [==============================] - 3s 5ms/step - loss: 0.6485 - categorical_accuracy: 0.6810 - val_loss: 0.5845 - val_categorical_accuracy: 0.7243
Epoch 3/20
630/630 [==============================] - 3s 4ms/step - loss: 0.6204 - categorical_accuracy: 0.7007 - val_loss: 0.5916 - val_categorical_accuracy: 0.7279
Epoch 4/20
630/630 [==============================] - 3s 4ms/step - loss: 0.6147 - categorical_accuracy: 0.7067 - val_loss: 0.5654 - val_categorical_accuracy: 0.7387
Epoch 5/20
630/630 [==============================] - 2s 4ms/step - loss: 0.6039 - categorical_accuracy: 0.7126 - val_loss: 0.5685 - val_categorical_accuracy: 0.7431
Epoch 6/20
630/630 [==============================] - 3s 4ms/step - loss: 0.6031 - categorical_accuracy: 0.7135 - val_loss: 0.5604 - val_categorical_accuracy: 0.7404
Epoc

# 7. Conclusão

<p> Com os testes realizados, foi possível ver com clareza que segmentar demais os targets atrapalha na predição da rede neural, onda a classificação sob a avaliação para 3 targets mostrou resultados bem melhores. A interpretação que podemos ter sobre isso é que o histograma entre o verso e a frente dos documentos são bem semelhantes, dificultando assim a avaliação da rede neural para diferenciá-los. </p> 
<p> Em relação às diferentes combinações de atributos, observamos que utilizar os histogramas do RGB + os 100 primeiros valores do histograma da matriz do HSV, bem como utilizar somente os 100 primeiros valores do histograma da matriz do HSV, apresentaram bons resultados para a classificação dos documentos, em contra partida, utilizar os histogramas do RGB não apresentou resultados tão bons. A partir disso, podemos perceber que os 100 primeiros valores do histograma da matriz do HSV foram bem significantes como abributos para a diferenciação dos documentos, pois tanto acompanhados dos histogramas do RGB como sozinhos apresentaram bons resultados. </p>
<p> Também foram testados diferentes arranjos de redes neurais, entretanto, a variedade de combinações sendo pouca por conta da simplicação do problema inicial, que era o nosso objetivo. Testamos redes neurais com 1 camada oculta densa, 2 camadas ocultas densas e 2 camadas ocultas densas com 1 de BatchNormalization. A utilização de 1 camada densa e 2 camadas densas apresentaram bons resultados, mas não mostraram diferenças significativas, em termos de acurácia geral. A utilização do BatchNormalization apenas acelerou o treinamento para convergir mais rápido, não apresentando aumento ou decremento na acurária.<p>
<p> Para a validação final dos modelos, por conta dos melhores resultados apresentados, escolhemos avaliar dois modelos: um com que os histogramas do RGB + os 100 primeiros valores do histograma da matriz do HSV; e o outro que utiliza somente os 100 primeiros valores do histograma da matriz do HSV. A arquitetura dos dois casos foi escolhida com base nos melhores resultados, as variações eram quase insignificantes, mas precisávamos de um critério para a escolha dos mesmos. As arquiteruras escolhidas foram: 2 camadas ocultas densas com 1 de BatchNormalization para o primeiro caso; e 1 camada oculta densa com 1 de BatchNormalization para o segundo caso.</p>